In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [7]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [8]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データを分析するにあたり、以下のポイントに焦点を当てます。

### 1. カテゴリー別売上
- **食品**、**衣服**、**日用品**の3つのカテゴリーが見受けられます。
- 各カテゴリーの売上、数量、利益（利益 = 売上 - 原価）を集計します。

### 2. 商品別のパフォーマンス
- 各商品の単価、売上、数量、原価を分析し、どの商品が最も売上に貢献しているかを評価します。
- 利益率（利益 / 売上）を計算し、どの商品が高い利益を上げるかを特定します。

### 3. 時間的な傾向
- 売上日別に売上のトレンドを可視化し、季節的なパターンや特定のキャンペーンの影響を把握します。
- 売上が特に高かった日や低かった日を特定し、その要因を分析します。

### 4. 数量の販売動向
- 各商品の数量販売の傾向を分析し、在庫管理や今後の発注戦略に役立てる提案をします。

### 5. 利益の最適化
- どのカテゴリーや商品が最も利益を上げているのかを把握し、そのデータを元に戦略的なアプローチを提案します（例: 高利益商品を重点的にマーケティングするなど）。

### 分析の方法
1. **ピボットテーブルを使用した集計**：各カテゴリーごとの売上、数量、利益を集約。
2. **可視化ツールの利用**：時間の経過と共に売上がどのように変化したかを見るためのグラフを作成。
3. **相関分析**：売上と数量、高単価の商品が数量に与える影響を分析。

### 提案
- 売上が特定の日に集中している場合、プロモーションを行うことが考えられます。
- 利益率が低い商品については、値上げやコスト削減のための見直しを検討する必要があります。
- 新商品の導入を検討する際は、売上が高くトレンドに合った商品特性を分析します。

このように分析することで、企業の売上向上に貢献できる洞察を得られます。ご希望の具体的な分析や視覚的な出力が必要であれば、お知らせください。


In [9]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0                   この売上データを分析するにあたり、以下のポイントに焦点を当てます。
1                                                    
2                                     ### 1. カテゴリー別売上
3           - **食品**、**衣服**、**日用品**の3つのカテゴリーが見受けられます。
4              - 各カテゴリーの売上、数量、利益（利益 = 売上 - 原価）を集計します。
5                                                    
6                                  ### 2. 商品別のパフォーマンス
7      - 各商品の単価、売上、数量、原価を分析し、どの商品が最も売上に貢献しているかを評価します。
8            - 利益率（利益 / 売上）を計算し、どの商品が高い利益を上げるかを特定します。
9                                                    
10                                      ### 3. 時間的な傾向
11   - 売上日別に売上のトレンドを可視化し、季節的なパターンや特定のキャンペーンの影響を把握します。
12                 - 売上が特に高かった日や低かった日を特定し、その要因を分析します。
13                                                   
14                                     ### 4. 数量の販売動向
15         - 各商品の数量販売の傾向を分析し、在庫管理や今後の発注戦略に役立てる提案をします。
16                                                   
17                          

In [10]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)